recap: previously in 2025:
- 

MATCHING GENEFORMER
- redo global normalization code; its not using the normalized data
- show entropy, etc. plots of rank matrix after normalization; shouldn't be the U-shape if its been normalized like that... right?
- need to accordingly normalize the input for the expression model too for comparison
- can do learned posenc, however this likely will not make a difference (carl says there is literature on it not affecting much)
- likely tho this won't be the primary cause of the issue

SEQUENTIAL VS. TABULAR
- verify what is tabular? what is sequential?
- from these results, we can conclude that it is being seen as sequential (specifically due to CLS embeddign plot)
- any way to make it more understandable? might not be relevant tho
- maybe try concat instead of addition and see if that changes anything lol

EVALUATING PREDICTIONS
- need *** plot variance by average expression (0-15) to see if it matches the distribution of the variance by rank plot
- can conclude from the results that the hypothesis is true, however what lea says is that the variance by rank plot is incorrect?

LINCS VS. TCGA
- can do TCGA because this is more similar to actual RNA seq we will be using later
- if there is a discrepancy between LINCS and TCGA resutls, then the result is readout-specific
- don't need to test EVERYTHING on tcga, just the results that are noteworthy from LINCS

INCORPORATING EXP VALUES
- carl asked: isn't the point of doing rank so that we don't have to use expression values??

for seb meeting:
- suggestions for CP presentation?
- what can i be doing better as a student/researcher? (maybe aritculation...)
- i feel behind

jan 18, 2025 - post-CP reorganization + hybrid research
- 

RE: PREV. RESULTS
- re-run all 4 results on trt dataset; make sure to match AE and TF # params
- clean up and separate files (esp AE)
- ?clean up GF testing code?

RE: HYBRID
- lit review on hybrid methods
- benchmarking tx eval

jan 23, 2025 - re-running reformatted code + more hybrid research
- 

WRT RESULTS

- rank_tf --> oni via sbatch (1ep) - 6hrs w/ batchsize of 210
    n_epochs = ?
    embed_dim = 128
    hidden_dim = 256
    n_heads = 2
    n_layers = 4
    drop_prob = 0.05
    lr = 0.001
    mask_ratio = 0.1
    - untrt -> 15min on smaug (128), 12min on oni (210) ? 

- exp_tf --> running rn on oni via sbatch (1ep)


- rank_ae --> 52min on smaug via sbatch (1ep)
    n_epochs = ?
    latent_1 = 
    latent_2 = 225
    latent_3 = 150
    embed_dim = 128
    drop_prob = 0.05
    lr = 0.001
    mask_ratio = 0.1

- exp_ae --> 1min on oni via sbatch (1ep)



WRT CODE

- ensure rank ae is done properly
- only thing is the "nothing"s that COULD be changed to work better with the file but not tooo important

WRT RESEARCH

- looking into: scVI, PCA, Harmony, HVG; how can what works with these be combined with our tf embeddings?

jan 27, 2026 - some notes on using calcul quebec
- 

ON LOGIN NODE:
- ssh chans@rorqual.alliancecan.ca
- julia --project=/home/chans/links/scratch/lincsv3 -e 'using Pkg; Pkg.instantiate(); Pkg.precompile()'
- (there are some issues with module stuff, may need module load cuda/12.9 julia/1.11.3?)

TO ALLOCATE:
- sbatch --time=00:30:00 simple_job.sh 
- salloc --time=1:0:0 --mem-per-cpu=3G --ntasks=1 --account=def-someuser  
    - salloc --time=00:30:00 --cpus-per-task=8 --gpus=nvidia_h100_80gb_hbm3_3g.40gb:1 --mem=62G

SLURM COMMANDS:
- sq: to check squeue for -u chans
- scontrol show job 632661 | grep: to see the path to executable/script
- scontrol show job 12345: for full info
- scontrol write batch_script 12345: for the file itself

NOHUP COMMANDS:
- ps -fp 12345: for script
- ps -p 12345 -o args: for wrap
- cat /proc/12345/cmdline: for all running processes
- ls -l /proc/12345/cwd: for folder

STEPS:
- ssh rorqual
- export JULIA_PKG_PRECOMPILE_AUTO=0
- instantiate pkgs, ensure everything is pkg.added
    - julia --project=/home/chans/links/scratch/lincsv3 -e 'using Pkg; Pkg.instantiate(); Pkg.precompile()'
- salloc
    - salloc --time=00:60:00 --nodes=1 --ntasks=1 --cpus-per-task=16 --mem=124G --gpus=h100:1
- module load julia/1.11.3 cuda/12.2 cudnn
- julia --project=/home/chans/links/scratch/lincsv3
    - cd to scripts/
    - julia --project=../
- ctrl + shift + enter to run by line in REPL

batchsizes w/ rorqual:
testing: 340

feb 5, 2026 - btiming/memory allocs :P
- 

given results.txt, #TODO:
- main issue: cpu-gpu movement
- also:
    - :gradient_calc, :loss_fwd, :m-transformer, :tf-attn
    - :get_ranks/errors is performing scalar iteration on CPU
        - moves the entire logit matrix to CPU and sorts preds for every single sample sequentially w/ loop; GPU idles
        - sol'n: instead of sorting on the CPU, calculating the rank of a target can be done via a broadcast so no need to sortperm

re: CPU/GPU data transfer:
- 36.5% of CUDA API: cudaMemcpyAsync (blocking bc of pageable memory)
    - sol'n: pinned array? (via CUDA.pin())
- 85.1% of GPU memory: device-to-host (gpu to cpu)
    - logits_masked and y_masked to CPU at the end of every batch in the test loop
    - sol'n: maybe dont need Flux.onecold?

https://cuda.juliagpu.org/stable/usage/memory/ 
http://fluxml.ai/Flux.jl/stable/guide/performance/ 


feb 9, 2026 - start merging embeddings!!!
- 

step 1: hybrid
- seb's approach: either:
    1. CLS token = PCA_emb --> transformer
    2. CLS token + PCA_emb --> transformer
    3. transformer --> trained CLS token --> PCA_emb --> transformer
- my approach: ranks_emb + encoder_emb --> transformer (reg/graph)

step 2: benchmark
- dataset: currently LINCS, could consider Lea's Tahoe pseudobulk
- tasks:
    - easy: identify cell line
    - medium: treated vs. untreated given profile
    - medium-hard: predict gene signature close to a phenotype
    - hard: signature activation; if a compound will induce some activation pathway
    - extra-hard: delta profiles (see Lea)
- via fine-tuning:
    - pretrain --> freeze --> add classifier head --> train --> take whole model --> train again = finetuning?

notes:
- log10(FLOPS) instead of epoch?
- emb_dim = 512
- see scFoundation embedding abalation in supp. material after having tried both hybrid approaches

step 1: approach #1 (CLS = PCA_emb for tf input)
- generate PCA vector for entire dataset
    - decide on #PCs
    - MultivariateStats.jl
    - project samples down to PC_dim
    - normalize x ~ N(0,1)
- architecture
    - need to project PCA vector size to transformer embedding input size
    - then embed the rest of the sequence aside from the CLS
- fwd pass
    - stick this CLS in front of batched embedding input

- how it works:
    - input: X_train (ranked matrix of n_genes x n_samples)
    - apply PCA for dim=64; pca_train is proj matrix of 64 x n_genes
    - masking applied to X_train
    - in training loop:
        - pca_train changed for each batch:
            - all mask_id converted into Float32 (0.0)
            - pca_train projected into batchsize (64 x n_genes --> 64 x batch_size)
            - z-score normalization
        - fwd pass:
            - move to gpu

            - input --> embedding (embed_dim, n_genes, batch_size)
            - pca_train --> projected to embed_dim (embed_dim, batch_size)
            - concat input + pca_train (embed_dim, n_genes + 1,  batch_size)

            - posenc (sin/cos) summation to input
            - pass thru transformer; PCA token at idx 1 aggregates info from all genes and allows all other gene tokens to see global variance (pca) context?
            - classification projects emb_dim to n_classes for logits prediction step
            - loss calculation: LCE
            
            - during backprop, gradient updates model.embedding separately and model.pca_proj separately
                - gradients do not flow into pca_train

step 1: approach #2 (CLS +/concat PCA_emb for tf input)
- as previously, take PCA vector but instead of just replacing CLS with the projected PCs, we can add or concat the PCA projection to the learned CLS (thus having 2 sets of learned weights as opposed to 1!)

- how it works:
    - input: X_train (ranked matrix of n_genes x n_samples)
    - apply PCA for dim=64; pca_train is proj matrix of 64 x n_genes
    - masking applied to X_train
    - in training loop:
        - pca_train changed for each batch:
            - all mask_id converted into Float32 (0.0)
            - pca_train projected into batchsize (64 x n_genes --> 64 x batch_size)
            - z-score normalization
        - fwd pass:
            - move to gpu

            - input --> embedding (embed_dim, n_genes, batch_size)
            - pca_train --> projected to embed_dim (embed_dim, batch_size)
            - cls_emb = cls_token.weight (learnable single vector initialized randomly to rep. cell-wise emb)
            - hybrid = pca_emb .+ cls_emb
            - concat input + hybrid_emb (embed_dim, n_genes + 1,  batch_size)

            - posenc (sin/cos) summation to input
            - pass thru transformer; hybrid token at idx 1 aggregates info from all genes and allows all other gene tokens to see global variance AND baseline ranked embedding context?
            - classification projects emb_dim to n_classes for logits prediction step
            - loss calculation: LCE
            
            - during backprop, gradient updates model.embedding separately and model.pca_proj and model.cls_token separately
                - gradients do not flow into pca_train
                - model.embedding: vector rep of specific genes
                - model.cls_token: vector rep of specific cells
                - model.pca_proj: vector rep of PCA statistics (shift from baseline cls)?

feb 11, 2026 - currently running:
-

using trt:
version 1: smaug 10ep w/ batchsize=128 + smaug 50ep w/ batchsize=128
version 2: kraken 1PM; n_ep=10, batch_size=42 on nohup (761928)

regular rtf: smaug 50ep w/ batchsize=128

feb 17, 2026 - 
- 

currently have:
version 2: kraken 1PM; n_ep=10, batch_size=42 on nohup (761928) - trt

running:
version 1: smaug 10ep w/ batchsize=128 - trt


if no time, run:

PARAMETERS:
########### kraken
dataset = untrt
masking_ratio = 0.15
mask_token_id = 979
batch_size = 64
n_epochs = 30
embed_dim = 128
latent_1 = 450
latent_2 = 225
latent_3 = 110
learning_rate = 0.001
ADDITIONAL NOTES: test run with new onehot input
run_time = 9 hours and 17 minutes
final_accuracy = 0.011015662717221921

for regular rtf, then v1 or v2 (whcihever u wanna explain)

feb 23, 2026 - reorganize, redo PCA implementation
- 

for reference, do:

*to create shell attached to sbatch (because regular sbatch is weird sometimes)
tmux new -s v1
srun --nodes=1 --ntasks=1 --cpus-per-task=16 --mem=90G --gres=gpu:V100:1 --time=10:00:00 --pty bash -i
nohup julia scripts/hybrid/version1.jl > v1_rtf_10ep.log 2>&1 &

*to reattach after ending session
tmux ls
tmux attach -t v1

*to delete session
tmux kill-session -t v1
tmux kill-server

to run for good plot comparison:
- rtf: trt, 50ep
    - 1ep: r(smaug-tmux)
- rae: trt, 50ep
    - 1ep: r(smaug-tmux)
- etf: trt, 50ep
    - 1ep: r(smaug-gpu1)
- eae: trt, 50ep
    - 1ep: 2mins (128)

for pca implementation. to run:
- no PCA no CLS (regular rank_ae.jl)
- PCA via concat (so PCA == CLS) (version 1)
- PCA via sum (exact same PCA per sample is summed to each row's embedding vector) (fix - version 2)

(since it was also said by quentin that CLS can be replaced with just taking the mean across all embedding vectors for each sample)

additional hybrids to run for good plot comparison:
- version 1: trt, 50ep
    - 1ep:
- version 2: trt, 50ep
    - 1ep: